In [2]:
import cv2
import numpy as np

In [46]:

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
 
def matchAndDisplayFeatures(im1, im2):

    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)
    min_d = min(matches,key=lambda x: x.distance)
    max_d = max(matches,key=lambda x: x.distance)
    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
    comparison = np.hstack((im1,im2))
    for i, match in enumerate(matches):
        x1,y1 = keypoints1[match.queryIdx].pt
        x2,y2 = keypoints2[match.trainIdx].pt
        x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
#         intensity = int(255. * (match.distance - min_d.distance) / max_d.distance)
        intensity = int(255 * (float(i)/len(matches)))
        intensity = 255 - intensity
        color = (intensity,0,0) if abs(y1 - y2) < 100 else (0,0,intensity)
        cv2.line(comparison, (x1,y1), (x2+im2.shape[1],y2), color, 2)
    
    

    return cv2.resize(comparison, (0,0), fx=0.5, fy=1)

In [47]:
cap = cv2.VideoCapture('../lecture.mp4')
_, old_frame = cap.read()

while 1:
    for i in range(24):
        if not cap.grab():
            break # TODO: only breaks from inner loop...
    ret, frame = cap.read()
    if not ret:
        break
        
    cv2.imshow('v', matchAndDisplayFeatures(old_frame,frame))
    old_frame = frame
    k = cv2.waitKey(1000) & 0xff
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()